### AMI Newmode timing & efficiency (early March 2024)

##### Eddie's remarks, Kevin's input, Anand's recoding Eddie's numbers.

Paul Goudfrooij:

This is what I’ve been able to find in NIRISS limitations documents with regard to requirements for subarray sizes, with the caveat that the rules might have changed over the last few years, although I would say that’s probably unlikely:

1. When NROWS ≤ 64 and NCOLS ≤ 64: NROWS x NCOLs = multiple of 64
2. When NROWS > 64 & NCOLS ≤ 512, OR NROWS ≤ 64 & 64 < NCOLS ≤512: NROWS = multiple of 4; NCOLS = multiple of 16
3. When NCOLS > 512: NCOLS = multiple of 64

Here, NCOLS and NROWS refer to columns and rows in the NATIVE (or “RAW”) detector coordinate system (where the SUB80 subarray is located at the right-hand edge of the detector array), not the DMS coordinate system. 

In [1]:
import numpy as np

%load_ext autoreload
%autoreload 2

ms = 1.0/1000
tpix = 1e-5

In [8]:
def magger(fluxrat):
    mag = 2.5*np.log10(fluxrat)
    return f'{ fluxrat:.1f} faster, {mag:.2f} mag improvement'
# print(magger(10), magger(100))
print(magger(16))

16.0 faster, 3.01 mag improvement


In [3]:
tf80x80 = (80+12) * (80+2) * tpix
print(f'tf_80x80 = {tf80x80/ms:.1f} ms')

tf36x36 = (36+12) * (36+4+2) * tpix
print(f'tf_36x36 = {tf36x36/ms:.1f} ms = {magger(tf80x80/tf36x36)}')
print()


tf_80x80 = 75.4 ms
tf_36x36 = 20.2 ms = 3.7 faster, 1.43 mag improvement



  

Now taking Kevin's example but porting that to the 36x36 case (that is, using superstripe to map the 36x36 area with say 9 36x4 stripes), you can do this but need refrows on each stripe at the very least because each stripe will be an independent integration with its separate preamp reset. Assuming the case Kevin suggested, with 4 refrows and 4 data rows on each stripe, the frametime of each 36x(4+4) stripe would be:

In [4]:
tf_9stripes_36x4 =  (36+12) * (((36/9.)+4)+2.) * tpix
print(f'tf_9stripes_36x4 = {tf_9stripes_36x4/ms:.1f} ms = {magger(tf80x80/tf_9stripes_36x4)}')
print()

tf_9stripes_36x4 = 4.8 ms = 15.7 faster, 2.99 mag improvement



Note that you would cover the full 36x36 science ROI with 9 superstripes, which would take 9x0.0048 = 0.0432 which is actually 75% faster than the current whole 80x80 frametime, but of course this is sequential and you have to take into account a separate reset and reads up the ramp on each integration. Your clock efficiency will be lower than the 80x80 case depending on the number of frames per int you want. However you do gain that factor of 15.7 in frametime. 

If the factor of 16 is more than you need, you could go with 36x8 stripes and cover a 36x40 ROI with 5 stripes, a factor of 13 shorter frametime. This would reduce your time between stripes clock overhead.


If you reduce the number of refrows per stripe from 4 to 2 (36x4=144 refpix samples per 144 datapix, vs 36x2=72 refpix samples per 144 datapix), your factor and clock efficiency both go up:   (((36+12) x (((36/9.)+2)+2.)) x 1e-5) = 0.00384000

In [5]:

tf_5stripes_36x8 = (( (36+12) * (((36/9.)+2)+2.))* tpix)
print(f'tf_5stripes_36x8 = {tf_5stripes_36x8/ms:.1f} ms = {magger(tf80x80/tf_5stripes_36x8)}')
print()


tf_5stripes_36x8 = 3.8 ms = 19.6 faster, 3.23 mag improvement



Which is a factor of 0.0754400 / 0.00384000 = 19.6 faster than the current 80x80. So if you can do a proper refsub with just 72 vs 144 refpix per stripe then you are good with just 2 refrows per stripe. You can use any 2 of the 4 bottom refrows here, so if for example the outside refrow is bad you can start one or two refrows in from the edge. Another possibility would be to take 2 refrows and two datarows. Not sure how that edge of the array is illuminated in AMI mode, but if it is effectively shadowed there is the possibility of using shadowed datarows instead of refrows for a 1/f corr. I've only played with this experimentally in NIRSpec for irs2. There are caveats - CRs and background for example. But datapix are a better match to the 1/f than the refpix. This would be graduate-level 1/f correction :)


One possible **caveat with striping your frame like this in AMI is charge migration from the areas just outside of each stripe**. 

In AMI you have a bright star here, which will be accumulating signal outside of your stripes, and if bright enough will start to "BF" spill charge into each stripe. This will roll over depending on your fast row resets between ints. I'd suggest the full 2048 rows between ints in this case. If you think this might cause problems you can also overlap your stripes by a row or two with superstriping. That way you could simply not use (throw away) the edges of the stripes where spilling would be worst. Or you might even be able to fashion a correction using the overlap and recover the S/N there. So there would be an efficiency hit with the extra reading of the overlap rows, but this can be done with super/substriping. Would be an interesting application of overlapping superstripes. 


---

**Just for the sake of completeness Eddie looks at FGSmode but it does not look interesting for AMI science**:  

Another way to reduce the frametime is to use FGS-mode rather than normal window mode. This is only available for windows that are 64x64 or less, and cuts the end of row buffer from 12 pixels times to 6 pixel times. It was my understanding that this wouldn't be allowed for science subarrays, but NIRISS is already using that for its smaller subarrays. This would only be a modest gain for 34x34, and on top of that you can't use super/supstriping in FGS mode (it needs the extra 6 pixel times per row for the math in the uC). So FGS mode would only be a slight improvement if you choose to stick with the 57x34 rectangle and no super/substriping. e.g. factor of 3.3 vs factor of 3.0 reduction in frametime.

---

**Efficiency**: Considering an example of the 9-stripe 36x((36/9)+2refrows) case, the frametime is a factor of 19 shorter than the current 80x80, but the overhead of taking 9 sequential stripe integrations eats into the wall-clock efficiency. To see what that looks like, I ran the numbers for an nframes=20 ramp case. As it turns out, the 9-stripe case is still wall-clock faster than the equivalent 80x80, but only by a factor of 2 (as opposed to, say, a factor of 19), and this is independent of the number of frames per int. So in reality this superstriped version is sampling your scene twice as often as the old 80x80 case. You can take twice as many exposures in the same amount of wall-clock time:

; ROI integration times
IDL> nframes=20
IDL> print,((nframes+1)x0.00384000)x9.   ; new
    0.725760
IDL> print,((nframes+1)x0.0754400)  ; old   
     1.58424

; ROI integration time, old/new
IDL> print,1.58424/0.725760
     2.18287

; frame time, old/new
IDL> print,0.0754400 / 0.00384000
     19.6458

Or to think of it another way, as long as your frame time ratio is >= the number of sequential stripes needed to cover the ROI, your temporal sampling is unharmed or improved. E.g. 19.6458 / 9 = 2.18287

That is to say, for your temporal stability requirement you are actually doing better with the new 9-striped mode than the old 80x80 case. A given science pixel in the ROI is sampled twice as often wall-clock time in the new regime. You're still taking a factor of 9/2.18 photon collection efficiency hit since you are integrating in serial rather than parallel. That plus any detector systematics induced by the serial striped collection (e.g. edge BF, others?) would be your trade-off. 

So if you wanted to add some overlapping rows to the stripes you could do so and still have an overall temporal sampling interval better than the old 80x80 case. Or use 4 refrows per stripe rather than the two in this example. Or both.
